#### Chargement des librairies

In [ ]:
import warnings
import pandas as pd
import numpy as np

#Visualisation
import itertools
import seaborn as sns
import matplotlib.pyplot as plt


#Modelisation
from sklearn.model_selection import train_test_split,StratifiedKFold,cross_validate,GridSearchCV,learning_curve,cross_val_predict
from sklearn.linear_model import LogisticRegression

#Performances
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score,roc_curve,f1_score,precision_score,recall_score,make_scorer

#### Fonctions de visualisation des métriques de performances

In [ ]:
#Affichage de la matrice de confusion
def plot_confusion_matrix(cm:np.array,classes:tuple,
                          normalize:bool = False,title:str ='Matrice de Confusion',
                          cmap = plt.cm.YlGn):
    '''
        Fonction permettant d'afficher une matrice de confusion
    '''
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    #print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Classes réelles')
    plt.xlabel('Classes prédites')
    plt.tight_layout()
    
def plot_roc_curve(y_true: pd.Series,
             y_proba: pd.Series,
             y_pred: pd.Series) :
    '''
        Fonction permettant d'afficher les courbes Roc des deux classes à prédire
    '''
    col = ['r','b'] #Couleur des courbes
    labels = ['Non embauché','embauché']
    roc_auc = roc_auc_score(y_true, y_pred)
    for i in range(len(col)):
        #Calcul de la courbe ROC des deux classes
        fpr, tpr, thr = roc_curve(y_true, y_proba[:,i], pos_label=i)
        #Affichage des courbes avec la couleur correspondante et la classe
        plt.plot(fpr, tpr,'-',c=col[i], label=labels[i])
        print(f"Moldèle (auc = {round(roc_auc,2)})")
    #Courbe ROC par défaut
    plt.plot([0,1],[0,1],'k--', label='hasard') 
    plt.xlabel(f"FPR Specificty")
    plt.ylabel(f"TPR (Sensitivity)")
    plt.legend(loc=4)
    plt.show()
    


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),minus=True):
    '''
        Fonction permettant d'afficher la courbe d'apprentissage et de validation en fonction du nombre d'observations qui varient
    '''
    if axes is None:
        _, axes = plt.subplots(1, 2, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True,shuffle=True,random_state=3)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)
    
    # Courbe d'apprentissage et de validation basé sur l'accuracy
    axes[0].grid()
    axes[0].fill_between(train_sizes,
                            train_scores_mean + train_scores_std,
                            train_scores_mean - train_scores_std,alpha=0.1,
                            color="r")
    axes[0].fill_between(train_sizes,test_scores_mean + test_scores_std,
                         test_scores_mean - test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")
        
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")
    
    # Courbe d'apprentissage et de validation basé sur 1 - l'accuracy
    axes[1].set_title(title+' cost')
    axes[1].grid()
    axes[1].fill_between(train_sizes,
                         1 - train_scores_mean + train_scores_std,
                         1 - train_scores_mean - train_scores_std,alpha=0.1,
                         color="r")
    axes[1].fill_between(train_sizes, 1 - test_scores_mean + test_scores_std,
                         1 - test_scores_mean - test_scores_std, alpha=0.1,
                         color="g")
    axes[1].plot(train_sizes, 1 - train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[1].plot(train_sizes, 1 - test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[1].legend(loc="best")
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("Cost = 1 - Score")
    #plt.show()
        
    df = pd.DataFrame({'train_sizes':train_sizes,
                        'train_score':train_scores_mean,
                        'train_std':train_scores_std,
                        'test_score':test_scores_mean,
                        'test_std':test_scores_std,
                        'train_size_per':np.round(train_sizes/X.shape[0],2)})
    
    return plt,df

def plot_performances(df: pd.DataFrame,
                      col: str,
                      metric_name: str,
                      color: str):
    '''
        Fonction permettant d'afficher un graphe avec l'ensemble des métriques calculées par modèle éxécutés
    '''
    bar_plot = go.Bar(y = df[col],
                      x = df[metric_name],
                      orientation = "h",
                      name = metric_name.replace('test_',''),
                      marker = dict(line = dict(width =.7),
                                    color = color)
                     )
    return bar_plot

#### Fonctions d'évaluations des performances des modèles

In [ ]:
def precision_custom(y_true: pd.Series, y_pred:pd.Series):
    '''
        Définition d'une métrique de precision personnalisé
    '''
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        try: 
            pr = precision_score(y_true, y_pred, average='weighted')
        except Warning:
            # Cas ou l'estimateur ne prédiraient aucun candidat embauché donc se retrouverait
            # à faire une division par 0 pour calculer la précision et le f1
            pr = precision_score(y_true, y_pred, average='weighted',zero_division=1)
        
    return pr

def recall_custom(y_true: pd.Series, y_pred:pd.Series):
    '''
        Définition d'une métrique de rappel personnalisé
    '''
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        try :
            re = recall_score(y_true, y_pred, average='weighted')
        except Warning: 
            re = recall_score(y_true, y_pred, average='weighted',zero_division=1)
        
    return re

def f_custom(y_true: pd.Series, y_pred:pd.Series):
    '''
        Définition d'une métrique de f1_score personnalisé
    '''
    with warnings.catch_warnings():
        warnings.filterwarnings('error')
        try:
            f_score = f1_score(y_true, y_pred, average='weighted')
        except Warning:
            f_score = f1_score(y_true, y_pred, average='weighted',zero_division=1)
        
    return f_score


def cross_val_monitor(model_tuple:tuple,
                      X: pd.DataFrame,
                      y: pd.Series,
                     folds: int):
    '''
        Fonction permettant d'éxécuter des modèles en se basant sur la méthode de cross-validation
    '''
    
    #Définition d'une grille de scoring custom
    scoring_custom = {'precision': make_scorer(precision_custom), 'recall': make_scorer(recall_custom),
           'f1_weighted': make_scorer(f_custom), 'accuracy': 'accuracy','roc_auc':'roc_auc'}
    #Extraction du nom du modèle ainsi que de l'estimateur
    model_name,model = model_tuple
    print(f"Cross-validation sur : {X.shape[0]} observations partagé en 5 partition avec le modèle : {model_name}")
    #Méthode de cross validation
    scores = cross_validate(model,X,y,
                            cv=folds,
                            scoring=scoring_custom,
                            return_train_score=True,
                            verbose=0,error_score=1)
    print(f"\n Affichage des scores: {model_name}")
    scoring = save_scores(scores,model_name)
    title = f"Courbe d'apprentissage et de validation : {model_name}"
    plot_learning_curve(model, title, X, y,
                        cv=folds, n_jobs=1)
    plt.show()
    
    print("Evaluation sur la totalité du jeu de données à l'aide de la fonction Cross val predict")
    model_pred = cross_val_predict(model,X,y)
    conf = confusion_matrix(y,model_pred)
    plot_confusion_matrix(conf,classes=['Non embauché','Embauché'])
    print('\n\n\n')
    
    return scoring
def save_scores(scores:dict,
                model_name:str,
                verbose:bool=True)->pd.DataFrame:
    '''
        Fonction permettant de sauvegarder les scores des différents classifiers éxécutés dans un dataframe
    '''
    if verbose:
        print(f"Scores training f1 weighted : {round(scores['train_f1_weighted'].mean(),3)} (+/- {round(scores['train_f1_weighted'].std(),3)})")
        print(f"Scores test f1 weighted : {round(scores['test_f1_weighted'].mean(),3)} (+/- {round(scores['test_f1_weighted'].std(),3)})")
    
        print(f"Scores training accuracy : {round(scores['train_accuracy'].mean(),3)} (+/- {round(scores['train_accuracy'].std(),3)})")
        print(f"Scores test accuracy : {round(scores['test_accuracy'].mean(),3)} (+/- {round(scores['test_accuracy'].std(),3)})")
    
        print(f"Scores training roc_auc : {round(scores['train_roc_auc'].mean(),3)} (+/- {round(scores['train_roc_auc'].std(),3)})")
        print(f"Scores test roc_auc : {round(scores['test_roc_auc'].mean(),3)} (+/- {round(scores['test_roc_auc'].std(),3)})")
    
        print(f"Scores training precision : {round(scores['train_precision'].mean(),3)} (+/- {round(scores['train_precision'].std(),3)})")
        print(f"Scores test precision : {round(scores['test_precision'].mean(),3)} (+/- {round(scores['test_precision'].std(),3)})")
    
        print(f"Scores training recall : {round(scores['train_recall'].mean(),3)} (+/- {round(scores['train_recall'].std(),3)})")
        print(f"Scores test recall : {round(scores['test_recall'].mean(),3)} (+/- {round(scores['test_recall'].std(),3)})")
    
    return pd.DataFrame({'test_f1_weighted':round(scores['test_f1_weighted'].mean(),2),
                       'test_f1_weighted_std':round(scores['test_f1_weighted'].std(),3),
                       'test_roc_auc':round(scores['test_roc_auc'].mean(),2),
                       'test_roc_auc_std':round(scores['test_roc_auc'].std(),3),
                       'test_precision':round(scores['test_precision'].mean(),2),
                       'test_precision_std':round(scores['test_precision'].std(),3),
                       'test_recall':round(scores['test_recall'].mean(),2),
                       'test_recall_std':round(scores['test_recall'].std(),3),
                       'test_accuracy':round(scores['test_accuracy'].mean(),2),
                       'test_accuracy_std':round(scores['test_accuracy'].std(),3),
                        'Model_name':pd.Series(model_name)})


def compute_performances(lstClf: list,X_train: pd.Series,
                         y_train: pd.Series,X_test: pd.Series,y_test: pd.Series):
    '''
        Fonction permettant de run une liste de classifieurs et de les évaluer selon certaines métriques de performances
    '''
    for model_name, clf in lstClf:
        clf.fit(X_train,y_train)
        print("\n" + model_name)
        y_pred = clf.predict(X_test)
        conf = confusion_matrix(y_test, y_pred)
        try:
            y_proba = clf.predict_proba(X_test)
            plot_roc_curve(y_true=y_test,y_proba=y_proba,y_pred=y_pred)
        except:
            print(f"L'estimateur n'a pas de probabilités\n")
        print ("Matrice de confusion :\n")
        plot_confusion_matrix(conf,classes=['Non Embauché','Embauché'])
        plt.show()
        print ("Resultats :\n")
        print (classification_report(y_test, y_pred))

### Dans ce script nous allons entreprendre la partie modélisation du problème (Machine Learning)

In [ ]:
data = pd.read_csv('../input/preprocessing-output/data_prepro.csv')

In [ ]:
data.head()

Comme nous l'avons vu, les classes que nous cherchons à prédire sont non balancées (Classe non-embauchés : 88% / Classe embauchés 11%) </br>
Cela pourrait induire en erreur les modèles de ML du fait qu'ils risquent de prédire tout candidats comme un candidat non embauché. </br>
Cependant certains algorithmes comme ceux basés sur des arbres de décisions ont cette capacité à pouvoir prédire des classes minoritairement représentées. </br>  Il existe notamment des méthodes de rééchantillonnage du jeu de données mais nous y reviendrons.

In [ ]:
X = data.drop(['embauche'],axis=1)
y = data.embauche.copy()

In [ ]:
y.value_counts()

In [ ]:
sns.countplot(y)

#### Méthodologie

Pour ce type de problématique, se baser uniquement sur un seul modèle serait une erreur car on pourrait potentiellement passer à coté</br> 
des modèles ayant des meilleures performances sur nos données. C'est pour cela que quelques modèles seront exécutés comme suit :

- Chaque modèle sera exécuté en utilisant la méthode de cross-validation. Cette méthode nous sera très utile afin de vérifier si notre modèle n'est pas en sur ou sous apprentissage. Mais en théorie, dans notre cas nous seront plus confrontés à des cas de surapprentissage due à la faible représentation de la classe des candidats embauchés.
- Pour chaque modèle, des métriques de performances (Précision, Recall, f1 score, roc_auc et l'accuracy) seront calculées et affichées.
Le choix de ces métriques s’est basé sur notre problématique de jeu de données non balancé. 
- Pour chaque modèle, une matrice de confusion sera calculée en se basant sur la méthode de cross-validation à l'aide de la fonction cross val predict. Cependant cette méthode n'est pas appropriée pour quantifier l'erreur de généralisation de notre modèle du fait qu'elle exige au modèle d'apprendre et prédire sur l'ensemble des données. Cela permet d'avoir une idée de la classification de notre modèle. </br>


#### Définition de notre stratégie de cross validation

Cette stratégie nous permet de partitionner notre jeu de données en 5 partitions et de s'entraîner sur 4 partitions représentant 80% des données et évaluer le modèle sur 1 partition représentant 20% des données. Ce processus est répété 5 fois avec 5 partitions différentes. De plus, chacune de ces partitions est stratifiée, c'est à dire qu’elles contiennent en termes de pourcentage environ le même échantillonnage que nos classes soient (88/11).

In [ ]:
skfolds = StratifiedKFold(n_splits=5)

#### Exemple de réflexion et de choix réalisé sur le paramétrage des algorithmes

Dans un premier temps nous testons une simple régression logistique, avec les hyperparamètres par défaut de sklearn. </br>
Remarques : 
- Le modèle a prédit l'ensemble des observations comme étant des candidats non embauchés. 
- Les mesures de précision, recall et f1 sont erronées, car la valeur du paramètre "zero_division" est fixé à 1. En faisant cela, toute division par 0 n'est pas soulevée comme un avertissement.
- Les graphique de gauche et de droite nous montre les courbes d'apprentissage et de validation du modèle. Celle de gauche nous montre le taux de bonne prédiction en fonction des différentes tailles de partitions sur lesquelles notre modèle s'entraine. Tandis que le graphe de droite nous montre le taux d'erreur (1 - accuracy). Certains sont plus à l'aise avec la lecture du graphique de droite c'est le plus représenté lorsque l'on parle de courbe d'apprentissage.

In [ ]:
lr = LogisticRegression(random_state=2)
Lr_tuple = ("LR",lr)
lr_res = cross_val_monitor(Lr_tuple,X,y,skfolds)

Nous sommes dans un cas de sur-apprentissage, il existe de nombreuses méthodes pour y palier. </br> 
L'une d'entre elle est de régulariser notre modèle. C'est à dire que nous allons contraindre notre modèle à trouver les valeurs des paramètres d'apprentissage dans un espace réduit. Ces mêmes paramètres qui dans notre cas tentent de maximiser le f1_score. </br> 
Pour ce faire nous utilisons la méthode de gridsearch nous permettant de tester différentes combinaisons de valeurs de paramètres définit dans une grille.

In [ ]:
param_grid = {'C': [1.0,10.0,100.0],
              'max_iter':[30,50,100],
              'class_weight': ['balanced']
             }
rg_cv = GridSearchCV(LogisticRegression(), param_grid, cv=skfolds, scoring='f1_weighted',verbose=0)
rg_cv.fit(X, y)
print(rg_cv.best_estimator_)

In [ ]:
print(f" le f1-score régularisé : {round(rg_cv.best_score_,2)}")

Remarques : 
- Après régularisation, nous obtenons un score f1 score de 63% ce qui dans un contexte de jeu de données non balancés n'est pas déplorable. Cependant en regardant l'accuracy , l'auc et le recall on s'aperçoit que les résultats sont très faible. Cela est surement dû au fait de notre régularisation sur le paramètre "class_weight" qui a pour but d'attribuer un poids plus lourd dans la prédiction de la classe minoritairement représentée. Dans notre cas les poids sont de : 0.56 pour la classe des candidats non embauchées et de 4.30 pour la classe des candidats embauchés.
- Nous voyons très bien, l'agissement de ce paramètre sur la matrice de confusion ou le nombre de FP est considérablement élevé. 
- Lorsqu'on regarde les courbes d'apprentissage elles traduisent un sous apprentissage évident. Du fait que le biais est très élevé environ 44%. Rajouter de la donnée n'est pas une solution car l'on voit que les deux courbes convergent à la dernière itération. Il faudrait soit alléger la régularisation de par un choix de poids de classe empiriques (class_weight) et/ou jouer sur les autres paramètres ou choisir un modèle plus complexe.

In [ ]:
Lr_tuple = ("LR",rg_cv.best_estimator_)
lr_res = cross_val_monitor(Lr_tuple,X,y,skfolds)

Scikit learn ne fournit pas beaucoup d'informations en sorties d'une régression logistique. </br>
Les coefficients ne sont donc pas très utiles car nous ne savons pas s'ils sont significatifs dans la prédiction d'un candidat embauché ou non. (Non présence de la p-value) </br>
Nous pouvons juste émettre l'hypothèse que si le coefficient de la variable est positif alors elle pousse notre modèle à prédire que notre candidat est embauché</br>
Une alternative consiste à utiliser la fonction logit du package stats.models qui fournit plus de sorties ou utiliser des packages aidant à interpréter les sorties de modèle de ML tels que lime,eli5 ou shap.</br>

In [ ]:
#Let's interpret the results
lr_coef = pd.DataFrame(np.concatenate([Lr_tuple[1].intercept_.reshape(-1,1),
                             Lr_tuple[1].coef_],axis=1),
             index = ["coef"],
             columns = ["constante"]+list(X.columns)).T

In [ ]:
lr_coef.sort_values(by='coef',ascending=False)

#### Choisissons un modèle plus complexe : L'arbre de décision

Ici nous avons régularisé le modèle en pénalisant la profondeur de l'arbre. Plus l'arbre est profond plus il prend des décision complexe et risque de faire du sur apprentissage. Mais cela n'était pas suffisant car nous avions des résultats trop justes en ce qui concerne notre classe minoritaire.

Remarques : 

- Les courbes d'apprentissage et de validation sont bonnes. Nous voyons bien sur le graphique de droite qu'au fur et à mesure que la partition réservée à l'entrainement augmente l'erreur sur le jeu de validation et d'entrainement décroît. Il y a encore un légère sur-apprentissage (expliqué par la variance entre les deux courbes) et l'épaisseur de la courbe de validation nous montre que les résultats sont assez dispersés mais le modèle donne de bonnes performances. 
- Cependant lorsqu'on jette un œil sur la matrice de confusion, on voit qu'on a toujours du mal à classifier les candidats embauchés. Nous pourrions mieux faire en pénalisant le poids de la classe minoritaire mais cela aura un effet immédiat sur les performances.

In [ ]:
from sklearn import tree

In [ ]:
Dt_tuple = ("DT",tree.DecisionTreeClassifier(max_depth=8,min_samples_leaf=7,min_samples_split=50,random_state=3))
Dt_res = cross_val_monitor(Dt_tuple,X,y,skfolds)

Dans cet exemple nous avons ajouté une pénalité sur le poids des classes lors des prédictions. </br> 
Au final nous perdons en précision car nous avons plus de FP, nos métriques de performances ont aussi baissés et nous observons que les résultats varient fortement ce qui montre une instabilité quant aux prédictions du modèle.

In [ ]:
Dt_regul = ("DT",tree.DecisionTreeClassifier(max_depth=8,min_samples_leaf=1,min_samples_split=10,class_weight='balanced',random_state=3))
Dt_res = cross_val_monitor(Dt_regul,X,y,skfolds)

#### KNN
Plus nous réduisons le nombre de voisins 'n_neighbors' plus le modèle sur-apprend

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_knn = ('Knn',KNeighborsClassifier(n_neighbors=20))
knn_res = cross_val_monitor(clf_knn,X,y,skfolds)

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier

In [ ]:
rf_tuple = ('Rf',RandomForestClassifier(random_state=3,max_depth=9,min_samples_leaf=3,
                                        min_samples_split=10,class_weight='balanced'))
rf_res = cross_val_monitor(rf_tuple,X,y,skfolds)

#### Extra Tree

In [ ]:
et_tuple = ('ET',ExtraTreesClassifier(random_state=3,max_depth=12,min_samples_leaf=5,
                                     min_samples_split=30,class_weight='balanced'))
et_res = cross_val_monitor(et_tuple,X,y,skfolds)

#### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_tuple = ('Gb',GradientBoostingClassifier(random_state=4,max_depth=3))
gb_res = cross_val_monitor(gb_tuple,X,y,skfolds)

#### XGboost
scale_pos_weight = total_negative_examples / total_positive_examples </br>
scale_pos_weight = 14856/1959 = 7.58

In [ ]:
import xgboost as xgb
xg_tuple = ('Xgb',xgb.XGBClassifier(random_state=19,scale_pos_weight=7.58))
xg_res = cross_val_monitor(xg_tuple,X,y,skfolds)

#### Catboost

In [ ]:
from catboost import CatBoostClassifier
clf_ct = ('CatB',CatBoostClassifier(random_state=5,logging_level='Silent',depth= 3,l2_leaf_reg = 3,learning_rate = 0.04))
ct_res = cross_val_monitor(clf_ct,X,y,skfolds)

Ce tableau nous permet de résumer les performances de chacun des modèles précédemment exécutés. </br>
Le choix va se faire sur le modèle qui maximise la classification des candidats embauchés</br>
Nous allons voir de manière plus détaillée ce qu'ils valent sur un jeu de données splitté en train et test.

In [ ]:
pd.set_option('display.float_format', '{:,.3f}'.format)
cm = sns.light_palette("green", as_cmap=True)
df_res = pd.concat([lr_res,Dt_res,knn_res,rf_res,gb_res,xg_res,ct_res,et_res],axis=0).reset_index(drop=True)
df_res = df_res.sort_values(by='test_f1_weighted',ascending=False)
s = df_res.style.background_gradient(cmap=cm)
s

In [ ]:
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
#initate plotly
pyoff.init_notebook_mode()

layout = go.Layout(dict(title = "Performances des modèles",
                        plot_bgcolor = "whitesmoke",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "Metriques",
                                     zerolinewidth=1,
                                     ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        height = 900
                       )
                  )


f1_score_w  = plot_performances(df_res,'Model_name',"test_f1_weighted","darkcyan")
roc_auc_s  = plot_performances(df_res,'Model_name','test_roc_auc',"darkgreen")
precision_s  = plot_performances(df_res,'Model_name','test_precision',"rebeccapurple")
recall_s  = plot_performances(df_res,'Model_name','test_recall',"slategray")
accuracy_s  = plot_performances(df_res,'Model_name','test_accuracy',"chocolate")
data = [f1_score_w,roc_auc_s,precision_s,recall_s,accuracy_s]
fig = go.Figure(data=data,layout=layout)
pyoff.iplot(fig)

#### Split du jeu de données en train et test

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,stratify=y,random_state=5)

In [ ]:
lst_clf =  [clf_ct,Dt_tuple,gb_tuple,et_tuple,xg_tuple,clf_knn,rf_tuple]

compute_performances(lst_clf,
            X_train,
            y_train,
            X_test,
            y_test)

##### And the winner is ... Random Forest !
Critère de choix du modèle :
- Le choix du modèle s'est réalisé entre Catboost et Random Forest. Catboost à l'avantage d'avoir des prédictions stable et une courbe d'apprentissage et de validations qui laisseraient entendre qu'avec plus de données les courbes pourraients "fitté" et palier ce légère sur-apprentissage sur les données d'entrainement.
- Intéressons nous aux performances des deux modèles sur la classe minoritaire. On s'aperçoit que Random Forest performe mieux à différents seuil de la courbe ROC (auc = 0.79) et qu'il a un meilleure score F1 (f1-score 0.52). De plus, il nous est possible d'améliorer ce score en trouvant le seuil optimal de décision pour la courbe de précision recall afin que le f1-score soit maximiser

Pour cela nous allons tracer la courbe de precision-recall du modèle Random Forest. Avant cela il nous faut fitté notre modèle sur le jeu de données d'entrainement et prédire les probabilités d'appartenir à la classe embauché sur le jeu de données Test.

In [ ]:
#Modèle random forest
clf_final = rf_tuple[1] 
clf_final.fit(X_train, y_train)
#On ne retiens que les proba d'appartenir à la classe embauché
y_proba = clf_final.predict_proba(X_test)[:,1]

In [ ]:
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

In [ ]:
# Affichage de la courbe precision-recall du modèle Random Forest
default = len(y_test[y_test==1]) / len(y_test)
plt.plot(recall, precision, marker='.', label='Precision-Recall')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

In [ ]:
# calcul du f1 score à différents seuils
fscore = (2 * precision * recall) / (precision + recall)
# localisation de l'index au score f1 le plus élevé
ind_max = np.argmax(fscore)
print(f"Seuil optimale = {round(thresholds[ind_max],3)}, F1-Score= {round(fscore[ind_max],3)}")

In [ ]:
# Affichage de la courbe precision-recall du modèle Random Forest avec le point
plt.plot(recall, precision, marker='.', label='Precision-Recall')
plt.scatter(recall[ind_max], precision[ind_max], marker='o', color='black', label='Best',s=90)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

Nous recalculons une matrice de confusion avec le seuil optimale et nous obtennons les résultats ci-dessous

In [ ]:
clf_final.fit(X_train, y_train)
#On ne retiens que les proba d'appartenir à la classe embauché
y_pred_bool = (clf_final.predict_proba(X_test)[:,1]>= thresholds[ind_max]).astype(bool)
y_pred = np.where(y_pred_bool==True,1,0)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(confusion_matrix(y_test, y_pred))

In [ ]:
precision = tp/(tp+fp)
rappel = tp/(tp+fn)

In [ ]:
print(f"rappel classe embauché = {rappel}\n précision classe embauché = {precision}\n f1-score classe embauché = {2*((precision*rappel)/(precision+rappel))}")

#### Recherchons les variables qui discriminent le plus nos candidats embauchés des non embauchés

Il est possible à partir d'un arbre de décision de calculer dans quelle mesure chaque caractéristique contribue à discriminer nos classes. </br>
Scikit learn nous pourvoit la méthode "features_importances" pour le cas de Random Forest. 
Remarques : 
- Les variables Note, salaire et disponibilité sont celles qui discrimiment au mieux nos classes. 

In [ ]:
model = clf_final.fit(X_train, y_train)
feature_imp = pd.Series(model.feature_importances_,index=X_train.columns).sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Score importance des variables')
plt.ylabel('Features')
plt.title("Importance des variables")
plt.legend()
plt.show()

Enregistrement du modèle Random Forest en prenant en compte le seuil optimisé

#### Piste d'améliorations :
- Utiliser des méthodes de réechantillonage. Ces méthodes d'undersampling ou d'oversampling permettent d'équilibrer les individus de chaque classes en y dupliquant des individus de la classe minoritaire pour le cas de l'oversampling ou en y supprimant des individus de la classe majoritaire pour le cas de l'undersampling. Mais il y a des méthodes plus avancés tels que SMOTE ou ADASYN qui permettent en autre de créer des données synthétiqueq basées sur des mesures de distances entre individus.
- Passer plus de temps sur la recherche de paramètres, qui pourraient réajuster les prédictions de la classe minoritaire pour le modèle Random Forest
- Utiliser des packages tels que LIME, ELI5 ou SHAP afin d'interprêter au mieux les résultats de notre modèle